In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%precision 3

'%.3f'

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# Online Retail.xlsx는 현재 디렉토리에 다운되어 있는 상태
# InvoiceNo는 일종의 청구 번호
# InvoiceNo가 동일한 상품은 한 번의 거래에서 함께 구매, 다시 말해서 동일한 InvoiceNo 세트가 하나의 거래
trans = pd.read_excel('/content/drive/MyDrive/DataScience/chap09/Online Retail.xlsx', sheet_name = 'Online Retail')
trans.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
# InvoiceNo의 첫 번째 문자가 5이면 정상적인 데이터, C는 취소, A는 불명확한 데이터
# InvoiceNo의 첫 번째 문자만 따로 추출해 'cancel_flg' 변수를 만듦
print(trans.shape)
trans['cancel_flg'] = trans['InvoiceNo'].map(lambda x:str(x)[0])
trans.groupby('cancel_flg').size()

(541909, 8)


cancel_flg
5    532618
A         3
C      9288
dtype: int64

In [ ]:
# CustomerID가 없는 데이터들이 많음
trans['CustomerID'].isnull().value_counts()

False    406829
True     135080
Name: CustomerID, dtype: int64

In [ ]:
# 정상 데이터(5)이면서, CustomerID가 있는 데이터만을 분석
trans = trans[(trans.cancel_flg == '5') & (trans['CustomerID'].notnull())]
trans

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,5
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,5
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,5
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,5
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,5
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,5


In [ ]:
trans['StockCode']

0         85123A
1          71053
2         84406B
3         84029G
4         84029E
           ...  
541904     22613
541905     22899
541906     23254
541907     23255
541908     22138
Name: StockCode, Length: 397924, dtype: object

In [ ]:
# 연관 규칙에 대해 알아보기 위해, 우선 구매 횟수(빈도) 기준 상위 5개 상품의 번호를 확인
# value_counts() 메소드를 통해 동일한 StockCode를 가지는 제품이 몇번 나타났는지 확인
trans['StockCode'].value_counts()

85123A    2035
22423     1724
85099B    1618
84879     1408
47566     1397
          ... 
90168        1
90169        1
90202A       1
90118        1
23843        1
Name: StockCode, Length: 3665, dtype: int64

In [ ]:
# 1위인 85123A와 3위의 85099B의 지지도 확인

# 모든 InvoiceNo를 추출하고 trans_all에 저장 (전체 거래 데이터에 해당)
trans_all = set(trans.InvoiceNo)

# 85123A를 구매한 거래 데이터를 추출하고 trans_a에 저장
trans_a = set(trans[trans['StockCode'] == '85123A'].InvoiceNo)

# 85099B를 구매한 거래 데이터를 추출하고 trans_b에 저장
trans_b = set(trans[trans['StockCode'] == '85099B'].InvoiceNo)

# len함수를 이용하여 구한 값이 거래 횟수에 해당
print(len(trans_a))
print(len(trans_b))

# 85123A와 85099B를 함께 구매한 거래 데이터를 추출하고 trans_ab에 저장
trans_ab = trans_a & trans_b
print(len(trans_ab))

1978
1600
252


In [ ]:
print(f'두 상품을 모두 포함하는 장바구니의 수: {len(trans_ab)}')
print(f'두 상품을 모두 포함하는 장바구니가 전체에서 차지하는 비율: {len(trans_ab) / len(trans_all):.3f}')

두 상품을 모두 포함하는 장바구니의 수: 252
두 상품을 모두 포함하는 장바구니가 전체에서 차지하는 비율: 0.014


In [ ]:
print(f'85123A를 포함하는 장바구니의 수: {len(trans_a)}')
print(f'85123A를 포함하는 장바구니가 전체에서 차지하는 비율: {len(trans_a) / len(trans_all):.3f}')

85123A를 포함하는 장바구니의 수: 1978
85123A를 포함하는 장바구니가 전체에서 차지하는 비율: 0.107


In [ ]:
# 85123A -> 85099B
print(f'신뢰도: {len(trans_ab) / len(trans_a):.3f}')

신뢰도: 0.127


In [ ]:
# 85099B -> 85123A
print(f'신뢰도: {len(trans_ab) / len(trans_b):.3f}')

신뢰도: 0.158


In [ ]:
support_b = len(trans_b) / len(trans_all)
confidence = len(trans_ab) / len(trans_a)

lift = confidence / support_b
print(f'향상도: {lift:.3f}')

향상도: 1.476


In [ ]:
# 9장 종합 문제 9-1
# 데이터의 수가 1000보다 큰 모든 상품을 대상으로 어떤 상품의 조합의 지지도가 가장 높은지 확인
from itertools import combinations

more_than_1000 = trans['StockCode'].value_counts()[trans['StockCode'].value_counts() > 1000]
max = 0
trans_all = set(trans.InvoiceNo)

for a, b in combinations(more_than_1000.index, 2):
    trans_a = set(trans[trans['StockCode'] == a].InvoiceNo)
    trans_b = set(trans[trans['StockCode'] == b].InvoiceNo)
    trans_ab = trans_a & trans_b
    support = len(trans_ab) / len(trans_all)
    if (support > max):
        max = support
        A = a
        B = b

print(f'상품 \'{A}\'와 상품 \'{B}\' 사이의 지지도가 {max:.5f}로 가장 큽니다.')

상품 '20725'와 상품 '22383' 사이의 지지도가 0.02800로 가장 큽니다.


In [ ]:
for a, b in combinations(more_than_1000.index, 2):
    trans_all = set(trans.InvoiceNo)
    trans_a = set(trans[trans['StockCode'] == a].InvoiceNo)
    trans_b = set(trans[trans['StockCode'] == b].InvoiceNo)
    trans_ab = trans_a & trans_b
    support = len(trans_ab) / len(trans_all)
    print(f'상품 \'{a}\'와 상품 \'{b}\' 사이의 지지도: {support:.5f}')

상품 '85123A'와 상품 '22423' 사이의 지지도: 0.01268
상품 '85123A'와 상품 '85099B' 사이의 지지도: 0.01360
상품 '85123A'와 상품 '84879' 사이의 지지도: 0.01462
상품 '85123A'와 상품 '47566' 사이의 지지도: 0.01505
상품 '85123A'와 상품 '20725' 사이의 지지도: 0.01230
상품 '85123A'와 상품 '22720' 사이의 지지도: 0.01063
상품 '85123A'와 상품 '20727' 사이의 지지도: 0.01014
상품 '85123A'와 상품 'POST' 사이의 지지도: 0.00092
상품 '85123A'와 상품 '23203' 사이의 지지도: 0.00723
상품 '85123A'와 상품 '22383' 사이의 지지도: 0.01111
상품 '85123A'와 상품 '21212' 사이의 지지도: 0.00798
상품 '85123A'와 상품 '22197' 사이의 지지도: 0.01036
상품 '85123A'와 상품 '23209' 사이의 지지도: 0.00847
상품 '85123A'와 상품 '23298' 사이의 지지도: 0.00917
상품 '85123A'와 상품 '22086' 사이의 지지도: 0.00615
상품 '85123A'와 상품 '22382' 사이의 지지도: 0.01009
상품 '22423'와 상품 '85099B' 사이의 지지도: 0.00858
상품 '22423'와 상품 '84879' 사이의 지지도: 0.01246
상품 '22423'와 상품 '47566' 사이의 지지도: 0.01235
상품 '22423'와 상품 '20725' 사이의 지지도: 0.00718
상품 '22423'와 상품 '22720' 사이의 지지도: 0.01176
상품 '22423'와 상품 '20727' 사이의 지지도: 0.00523
상품 '22423'와 상품 'POST' 사이의 지지도: 0.00842
상품 '22423'와 상품 '23203' 사이의 지지도: 0.00561
상품 '22423'와 상품 '22383' 사

In [ ]:
more_than_1000

85123A    2035
22423     1724
85099B    1618
84879     1408
47566     1397
20725     1317
22720     1159
20727     1105
POST      1099
23203     1098
22383     1083
21212     1068
22197     1068
23209     1029
23298     1024
22086     1019
22382     1008
Name: StockCode, dtype: int64

In [ ]:
# 향상도 버전
more_than_1000 = trans['StockCode'].value_counts()[trans['StockCode'].value_counts() > 1000]

max = 0

trans_all = set(trans.InvoiceNo)

for a, b in combinations(more_than_1000.index, 2):
    trans_a = set(trans[trans['StockCode'] == a].InvoiceNo)
    trans_b = set(trans[trans['StockCode'] == b].InvoiceNo)
    trans_ab = trans_a & trans_b
    lift = (len(trans_ab) * len(trans_all)) / (len(trans_a) * len(trans_b))
    if (lift > max):
        max = lift
        A = a
        B = b

print(f'상품 \'{A}\'와 상품 \'{B}\' 사이의 향상도가 {max:.5f}로 가장 큽니다.')

상품 '22383'와 상품 '22382' 사이의 향상도가 8.04049로 가장 큽니다.


In [19]:
# 교재의 답안
indexer = trans.StockCode.value_counts() > 1000
Items = trans.StockCode.value_counts()[indexer.index[indexer]].index
trans_all = set(trans.InvoiceNo)

results = {}
for element in combinations(Items, 2):
    trans_a = set(trans[trans['StockCode'] == element[0]].InvoiceNo)
    trans_b = set(trans[trans['StockCode'] == element[1]].InvoiceNo)
    trans_ab = trans_a & trans_b
    support = len(trans_ab) / len(trans_all)
    results[element] = support
    
maxKey = max([(v, k) for k, v in results.items()])[1]
print(f'상품 \'{maxKey[0]}\'와 상품 \'{maxKey[1]}\' 사이의 지지도가 {results[maxKey]:.5f}로 가장 큽니다.')

상품 '85099B'와 상품 '22386' 사이의 지지도가 0.03216로 가장 큽니다.


In [ ]:
indexer

85123A     True
22423      True
85099B     True
84879      True
47566      True
          ...  
90168     False
90169     False
90202A    False
90118     False
23843     False
Name: StockCode, Length: 3665, dtype: bool

In [ ]:
indexer.index

Index(['85123A',    22423, '85099B',    84879,    47566,    20725,    22720,
          20727,   'POST',    23203,
       ...
       '90039A', '90039B',    21268, '90059A', '90059D',    90168,    90169,
       '90202A',    90118,    23843],
      dtype='object', length=3665)

In [ ]:
indexer.index[indexer]

Index(['85123A',    22423, '85099B',    84879,    47566,    20725,    22720,
          20727,   'POST',    23203,    22383,    21212,    22197,    23209,
          23298,    22086,    22382],
      dtype='object')

In [ ]:
Items

Index(['85123A',    22423, '85099B',    84879,    47566,    20725,    22720,
          20727,   'POST',    23203,    22383,    21212,    22197,    23209,
          23298,    22086,    22382],
      dtype='object')

In [ ]:
more_than_1000.index

Index(['85123A',    22423, '85099B',    84879,    47566,    20725,    22720,
          20727,   'POST',    23203,    22383,    21212,    22197,    23209,
          23298,    22086,    22382],
      dtype='object')

In [ ]:
trans[trans['StockCode'] == '85123A']

In [ ]:
results.items()

dict_items([(('85123A', 22423), 0.01267803193785067), (('85123A', '85099B'), 0.013595166163141994), (('85123A', 84879), 0.014620198532585239), (('85123A', 47566), 0.015051791109192922), (('85123A', 20725), 0.012300388433318947), (('85123A', 22720), 0.010627967198964177), (('85123A', 20727), 0.010142425550280535), (('85123A', 'POST'), 0.000917134225291325), (('85123A', 23203), 0.007229175658178679), (('85123A', 22383), 0.011113508847647821), (('85123A', 21212), 0.007984462667242124), (('85123A', 22197), 0.010358221838584376), (('85123A', 23209), 0.008470004315925766), (('85123A', 23298), 0.00917134225291325), (('85123A', 22086), 0.006150194216659474), (('85123A', 22382), 0.010088476478204576), ((22423, '85099B'), 0.008577902460077686), ((22423, 84879), 0.012462235649546828), ((22423, 47566), 0.012354337505394906), ((22423, 20725), 0.007175226586102719), ((22423, 22720), 0.011760897712559344), ((22423, 20727), 0.005233059991368148), ((22423, 'POST'), 0.008416055243849805), ((22423, 23203

In [ ]:
max([(v, k) for k, v in results.items()])

(0.028, (20725, 22383))